# LCEL (대화내용 기억하기): 메모리 추가

임의의 체인에 메모리를 추가하는 방법을 보여줍니다. 현재 메모리 클래스를 사용할 수 있지만 수동으로 연결해야 합니다


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
# from operator import itemgetter
# from langchain.memory import ConversationBufferMemory
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.runnables import RunnableLambda, RunnablePassthrough
# from langchain_openai import ChatOpenAI


# # ChatOpenAI 모델을 초기화합니다.
# model = ChatOpenAI()

# # 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful chatbot"),
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human", "{input}"),
#     ]
# )

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory


# 1. 모델
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 2. 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful chatbot"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])





대화내용을 저장할 메모리인 `ConversationBufferMemory` 생성하고 `return_messages` 매개변수를 `True`로 설정하여, 생성된 인스턴스가 메시지를 반환하도록 합니다.

- `memory_key` 설정: 추후 Chain 의 `prompt` 안에 대입될 key 입니다. 변경하여 사용할 수 있습니다.


In [5]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
# memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
# 3. 히스토리(메모리 대체)
store = {}

def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

저장된 대화기록을 확인합니다. 아직 저장하지 않았으므로, 대화기록은 비어 있습니다.


In [6]:
# memory.load_memory_variables({})  # 메모리 변수를 빈 딕셔너리로 초기화합니다.

# 최신 버전에서 필요한 형태로 래핑한 함수
# 4. 전체 체인
chain = RunnableWithMessageHistory(
    prompt | model,
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)

`RunnablePassthrough.assign`을 사용하여 `chat_history` 변수에 `memory.load_memory_variables` 함수의 결과를 할당하고, 이 결과에서 `chat_history` 키에 해당하는 값을 추출합니다.


In [7]:
# runnable = RunnablePassthrough.assign(
#     chat_history=RunnableLambda(memory.load_memory_variables)
#     | itemgetter("chat_history")  # memory_key 와 동일하게 입력합니다.
# )

#위에 코드와 통합


In [8]:
# runnable.invoke({"input": "hi"})

chain.invoke(
    {"input": "hi"},
    config={"configurable": {"session_id": "session1"}}
)


AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 17, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CcU3DZHRgr1u7vPdOFzJ2icmqd53u', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--47a9128a-3ee0-4a4d-9aea-efa77f64f1b9-0', usage_metadata={'input_tokens': 17, 'output_tokens': 9, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
# runnable.invoke({"input": "hi"})

chain.invoke(
    {"input": "hi"},
    config={"configurable": {"session_id": "session1"}}
)


AIMessage(content='Hi again! How can I help you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 35, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CcU3Fa84FfLWitLs6LYB8rHKU11De', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--52c1b495-c1a0-4411-ae3c-e2eff70dfbda-0', usage_metadata={'input_tokens': 35, 'output_tokens': 10, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful chatbot"),
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human", "{input}"),
#     ]
# ) #위에서 chain을 구성하기 위해 정의

`runnable` 에 첫 번째 대화를 시작합니다.

- `input`: 사용자 입력 대화가 전달됩니다.
- `chat_history`: 대화 기록이 전달됩니다.


In [ ]:
#runnable.invoke({"input": "hi!"})

In [ ]:
#chain = runnable | prompt | model 

첫 번째 대화를 진행합니다.


In [47]:
# chain 객체의 invoke 메서드를 사용하여 입력에 대한 응답을 생성합니다.
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 테디입니다."}, 
                        config={"configurable": {"session_id": "session1"}})
print(response.content)  # 생성된 응답을 출력합니다.

만나서 반갑습니다, 테디님! 어떻게 도와드릴까요?


In [ ]:
history = get_history("session1")
for msg in history.messages:
    print(msg)



content='hi' additional_kwargs={} response_metadata={}
content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 17, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CcU3DZHRgr1u7vPdOFzJ2icmqd53u', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--47a9128a-3ee0-4a4d-9aea-efa77f64f1b9-0' usage_metadata={'input_tokens': 17, 'output_tokens': 9, 'total_tokens': 26, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content='hi' additional_kwargs={} response_metadata={}
content='Hi again! How can I help you today?' additional_kwargs={'ref

`memory.save_context` 함수는 입력 데이터(`inputs`)와 응답 내용(`response.content`)을 메모리에 저장하는 역할을 합니다. 이는 AI 모델의 학습 과정에서 현재 상태를 기록하거나, 사용자의 요청과 시스템의 응답을 추적하는 데 사용될 수 있습니다.


이름을 기억하고 있는지 추가 질의합니다.


In [48]:
# 이름을 기억하고 있는지 추가 질의합니다.
history = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"},
                        config={"configurable": {"session_id": "session1"}})
# 답변을 출력합니다.
print(history.content)

네, 테디님이라고 하셨습니다! 어떻게 도와드릴까요?


## 커스텀 ConversationChain 구현 예시

In [ ]:
# from operator import itemgetter
# from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from langchain_core.runnables import RunnableLambda, RunnablePassthrough, Runnable
# from langchain_openai import ChatOpenAI
# from langchain_core.output_parsers import StrOutputParser

# # ChatOpenAI 모델을 초기화합니다.
# llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)

# # 대화형 프롬프트를 생성합니다. 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", "You are a helpful chatbot"),
#         MessagesPlaceholder(variable_name="chat_history"),
#         ("human", "{input}"),
#     ]
# )

# # 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
# memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

from langchain_core.runnables import Runnable
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage



In [36]:
from langchain_core.runnables import Runnable
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage


class MyConversationChain(Runnable):


    def __init__(self, llm, prompt, input_key="input",
                 max_messages_before_summary=10):
        self.llm = llm
        self.prompt = prompt
        self.input_key = input_key
        self.max_messages_before_summary = max_messages_before_summary

        # 세션별 대화기록 저장소
        self.store = {}
        self.summary_store = {}  # 요약 저장소

        def get_history(session_id):
            if session_id not in self.store:
                self.store[session_id] = InMemoryChatMessageHistory()
                self.summary_store[session_id] = ""  # summary 초기화
            return self.store[session_id]

        # 기본 conversation chain
        self.base_chain = (
            self.prompt
            | self.llm
            | StrOutputParser()
        )

        # history 자동 관리 runnable
        self.chain = RunnableWithMessageHistory(
            self.base_chain,
            get_history,
            input_messages_key=self.input_key,
            history_messages_key="history",
        )

        # 요약용 프롬프트
        self.summary_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "You summarize a conversation between a human and an AI."),
                ("human",
                 "Existing summary:\n{existing_summary}\n\n"
                 "New messages:\n{new_messages}\n\n"
                 "Write an updated compressed summary in Korean.")
            ]
        )

    def summarize(self, session_id):
        #요약을 생성하고 summary_store 갱신
        history = self.store[session_id].messages
        existing_summary = self.summary_store.get(session_id, "")

        # 히스토리를 문자열로 단순 변환
        text_blocks = []
        for m in history:
            role = "Human" if isinstance(m, HumanMessage) else "AI"
            text_blocks.append(f"{role}: {m.content}")
        new_text = "\n".join(text_blocks)

        chain = self.summary_prompt | self.llm | StrOutputParser()

        new_summary = chain.invoke(
            {
                "existing_summary": existing_summary or "(요약 없음)",
                "new_messages": new_text,
            }
        )

        # summary 갱신
        self.summary_store[session_id] = new_summary

        # history 축소 (최근 몇 개만 유지)
        last_few = history[-4:]
        history.clear()
        history.extend(last_few)

    def invoke(self, query, config):
        #요약 로직 포함한 대화 처리

        session_id = config["configurable"]["session_id"]

        # === 1) 대화 실행 ===
        response = self.chain.invoke(
            {self.input_key: query},
            config=config
        )

        # === 2) 요약 조건 체크 ===
        history = self.store[session_id]
        if len(history.messages) > self.max_messages_before_summary:
            self.summarize(session_id)

        return response
    
    
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot."),
        MessagesPlaceholder("history"),
        ("human", "{input}")
    ]
    )

conversation_chain = MyConversationChain(llm, prompt)





In [42]:
conversation_chain.invoke("안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.",
                          config={"configurable": {"session_id": "session1"}})

'안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [43]:
conversation_chain.invoke(
    "제 이름이 뭐라고요?",
    config={"configurable": {"session_id": "session1"}}
)


'테디님이라고 하셨어요! 어떻게 도와드릴까요?'

In [44]:
conversation_chain.invoke("앞으로는 영어로만 답변해주세요 알겠어요?", 
                          config={"configurable": {"session_id": "session1"}})

'Understood! I will respond in English from now on. How can I assist you today?'

In [45]:
conversation_chain.invoke("제 이름을 다시 한 번 말해주세요",                     
                          config={"configurable": {"session_id": "session1"}})

'Your name is Teddy. How can I assist you further?'

In [46]:
# conversation_chain.memory.load_memory_variables({})["chat_history"]


session_id = "session1"

history = conversation_chain.store[session_id]  # ← chain → conversation_chain 로 수정
messages = history.messages

for msg in messages:
    print(msg)

content='안녕하세요? 만나서 반갑습니다. 제 이름은 테디 입니다.' additional_kwargs={} response_metadata={}
content='안녕하세요, 테디님! 만나서 반갑습니다. 어떻게 도와드릴까요?' additional_kwargs={} response_metadata={}
content='제 이름이 뭐라고요?' additional_kwargs={} response_metadata={}
content='테디님이라고 하셨어요! 어떻게 도와드릴까요?' additional_kwargs={} response_metadata={}
content='앞으로는 영어로만 답변해주세요 알겠어요?' additional_kwargs={} response_metadata={}
content='Understood! I will respond in English from now on. How can I assist you today?' additional_kwargs={} response_metadata={}
content='제 이름을 다시 한 번 말해주세요' additional_kwargs={} response_metadata={}
content='Your name is Teddy. How can I assist you further?' additional_kwargs={} response_metadata={}
